In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You canq write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/delaysenc/delays20enc.csv', index_col=False, warn_bad_lines=True, error_bad_lines=False)
df=df.drop(['Unnamed: 0'], axis=1)
df = df.sample(frac =.02).reset_index().drop(['index'], axis=1)
df.head()

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
X,y = df.loc[:, df.columns != 'ArrDelay'],df.loc[:, df.columns == 'ArrDelay']

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.dummy import DummyClassifier

# Using the Dummy Classifier to set the lowest accuracy and therefore find the best model
dc = DummyClassifier(random_state=42) # define the model

# Train/fit the model to the data
dc.fit(X_train, y_train)

# Apply the model to the testing data to get a new set of predictions
# and show the accuracy score
prediction = dc.predict(X_test)
print('Arrival delay Dummy Classifier Accuracy:', metrics.accuracy_score(prediction, y_test))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Support Vector Classification
kernel= ['linear', 'poly', 'rbf', 'sigmoid']

skf10 = StratifiedKFold(n_splits=10)
skf5 = StratifiedKFold(n_splits=5)

for k in kernel:
    clf = svm.SVC(kernel=k, random_state=42, gamma='auto')
    scores = cross_val_score(clf, X, y, cv=skf5, scoring='accuracy').mean()
    print(k, 'accuracy with 5 splits: ',scores)
    scores = cross_val_score(clf, X, y, cv=skf10, scoring='accuracy').mean()
    print(k, 'accuracy with 10 splits: ',scores)


In [ ]:
# Kneighbors Classifier
skf = StratifiedKFold(n_splits=5)

k_range = list(range(5, 35))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=skf, scoring='accuracy')
    k_scores.append(scores.mean())
    print(k,'range: ',k_scores)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the value of K for KNN (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')

In [ ]:
# 10-fold cross-validation with logistic regression
from sklearn.linear_model import LogisticRegression
solv=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

for ss in solv:
    logreg = LogisticRegression(solver=ss)
    print(ss,'10:   ',cross_val_score(logreg, X, y, cv=skf10, scoring='accuracy').mean())
    print(ss,'5:    ',cross_val_score(logreg, X, y, cv=skf5, scoring='accuracy').mean())

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
skf10 = StratifiedKFold(n_splits=10)
scores = -cross_val_score(lm, X, y, cv=skf10, scoring='neg_mean_squared_error')
print(scores)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

depth=list(range(3, 13))

for dd in depth:
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=dd, random_state=42)
    print('Entropy, max depth ',dd,': ',cross_val_score(clf, X, y, cv=skf10, scoring='accuracy').mean())
    
    clf = DecisionTreeClassifier(criterion="gini", max_depth=dd, random_state=42)
    print('Gini, max depth ',dd,': ',cross_val_score(clf, X, y, cv=skf10, scoring='accuracy').mean())

In [ ]:
from sklearn.neural_network import MLPClassifier
#MLPClassifier(alpha=1, max_iter=1000)

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
# GaussianProcessClassifier(1.0 * RBF(1.0))

from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [ ]:
#Initializing the MLPClassifier

lst=[(20,10,5),(150,100,50),(64,64,64)]
iter=(500,200,50)
cvs=(5,10)

for hls in lst:
    for it in iter:
        for cv in cvs:
            mlp = MLPClassifier(hidden_layer_sizes=hls, max_iter=it,activation = 'relu',solver='adam',random_state=1)
            print ('hidden layers: ',hls,' max iterations: ', it, ' number of folds: ',cv)
            print('mean accuracy: ',cross_val_score(mlp, X, y, cv=cv, scoring='accuracy').mean())


In [ ]:
# Gaussian Process Classifier
gpc = GaussianProcessClassifier(kernel=1*RBF(1.0))
